## Question Answering System
### Team 6 
#### Amit Raj, Rakesh Subramani Kaleeshwaran, Abinash Vasudevan 

In [1]:
## Importing libraries

import wikipedia
import lxml
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.chunk import tree2conlltags                          
from nltk.corpus import stopwords                              
from datetime import datetime
import sys
import os
import re
import nltk

In [2]:
## Defining the main function

def main():
    try:
        file=open(sys.argv[1], "w+")
        print("Hello! This is a question answering system by Team 6. Please type your query. If you are done, Please type 'Exit' to leave the program.")  
        
        while True:
            ans=""
            wiki_ans=""
            person=False
            ques=input("Hello! Please enter your query?\n ")
            file.write(ques + "\n")
            
            if ques == "exit":
                exit()
                break
            if ques != "" and ques is not None:
                ques=ques.title()
                nertree=nertree_generation(ques)
                regmatch=ques_processing(ques)
                if regmatch != "":
                    if ("Where" in ques or "What" in ques):
                        wiki_ans=wiki_data(regmatch, file)
                        similarity=re.search(r'\d{4}', wiki_ans)
                        if similarity is not None:
                            wiki_ans= ""
                            
                    if('B-PERSON' or 'I-PERSON') in nertree[2]:
                        person=True
                    elif ('I-PERSON' or 'B-PERSON') in nertree[2]:
                        person = True
                    if "Who" in ques and person:
                        wiki_ans=wiki_data(regmatch, file)
                    
                    if("When" in ques or "Age" in ques) and person:
                        wiki_ans=wiki_data(regmatch, file)
                        ques_result = query_processing(wiki_ans)
                        if ques_result != "":
                            if "Age" in ques:
                                similarity=re.search(r'\d{4}', ques_result)
                                if similarity is not None:
                                    ques_result=datetime.now().year - int(similarity.group())
                            append_user_res=changing_regx_patt(ques, ques_result)
                            if append_user_res is not None:
                                ans=append_user_res
                        else:
                            ans=""
                    else:
                        ans=wiki_ans
                else:
                    ans=""
            else:
                print("Enter a valid question")
            if ans =="":
                print("Oops ! Answer not found")
            else:
                print(ans)
            file.write(ans + "\n")
            
    except Exception as GeneralException:
        print(GeneralException)
    finally:
        file.close()
                        
            

In [3]:
#### Processing questions

def ques_processing(ques):
    query_exp=""
    patterns = [
        
        [r'Who (Is|Was) (.+)',
         ["{0}"]],
        
        [r'Where (Is|Was) (.+)',
         ["{0}"]],

        [r'What (Is|Was) (.+)',
         ["{0}"]],
        
         [r'When (Is|Was) (.+)',
         ["{0}"]],
        
        [r'Why (Is|Was) (.+)',
         ["{0}"]],

        [r'What (Is|Was) (.+) Age',
         ["{0}"]],

        [r'When (Is|Was) (.+) Birthday',
         ["{0}"]]

    ]
    
## Retrieving query subject

    for regx_patt, queries in patterns:
        similarity=re.match(regx_patt, ques.rstrip('.!?'))
        if similarity:
            query_exp=similarity.groups()[1]
            
    return query_exp

In [4]:
## Retriving summary from Wikipedia

def wiki_data(query,file):
    try:
        file.write(query + "\n")
        ques_result=wikipedia.summary(query, sentences=1)
        file.write(ques_result + "\n")
    except Exception as GeneralException:
        ques_result=""
    return ques_result
        
        

In [5]:
## Using regular expressions for processing the entered queries

def query_processing(wiki_ans):
    query_exp=""
    patterns=[

        [r'[\d]+\s(\w)+\s[\d]+',
         ["{0}"]],

        [r'(?<=\().(\w+).(\w+,).(\w+)',
         ["{0}"]]
        
    ]
    
    for regx_patt, queries in patterns:
        similarity=re.search(regx_patt, wiki_ans)
        if similarity is not None:
            query_exp=similarity.group()
    return query_exp

In [6]:
## Extracting answers using meaningful sentences

def changing_regx_patt(user_Question,ques_result):
    user_Question=user_Question.rstrip('.!?')
    ques=re.sub(r'When (Is|Was) (.+) Born', r'\2 Was Born On ', user_Question)
    if ques != user_Question:
        return ques + str(ques_result) + '.'
    ques=re.sub(r'When (Is|Was) (.+) Birthday', r'\2 Birthday Is On ', user_Question)
    if ques != user_Question:
        ques= ques + ques_result
        ques= re.sub(r'\d{4}', '', ques)
        return ques + '.'
    ques = re.sub(r'What (Is|Was) (.+) Age', r'\2 Age Is ', user_Question)
    if ques != user_Question:
        return ques + str(ques_result) + '.'

    ### function for retrieving birthday , age of person form wikipedia

In [7]:
## Exit the program

def exit():
    print("Thankyou! Hope you got your queries answered.")

In [8]:
## Generating ntrees

def nertree_generation(ques):
    ques_tokens=word_tokenize(ques)
    stopWords=set(stopwords.words('english'))
    stopWords.add('?')
    stopWords.add('!')
    array_ques=[]
    for word in ques_tokens:
        if word not in stopWords:
            array_ques.append(word)
    nertree=tree2conlltags(ne_chunk(pos_tag(word_tokenize(ques))))
    return nertree

### Calling main function  

In [ ]:
main()

Hello! This is a question answering system by Team 6. Please type your query. If you are done, Please type 'Exit' to leave the program.
Hello! Please enter your query?
 who is donald trump?
Donald John Trump (born June 14, 1946) is an American media personality and businessman who served as the 45th president of the United States from 2017 to 2021.
